<a href="https://colab.research.google.com/github/GustasAndre/Campanha_MKT/blob/main/desafio_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Desafio: Construindo um modelo de Regressão para marketing


In [64]:
#Importar as bibliotecas necessárias para o case
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [ ]:
#Importar dados CSV.
mkt = pd.read_csv("/content/drive/MyDrive/MKT.csv")
mkt.head()

In [ ]:
# Criar a coluna 'Total_de_anuncios' somando as colunas 'youtube', 'facebook' e 'newspaper'
mkt['Total_de_anuncios'] = mkt[['youtube', 'facebook', 'newspaper']].sum(axis=1)

# Reorganizar as colunas
mkt = mkt[['youtube', 'facebook', 'newspaper', 'Total_de_anuncios', 'sales']]

# Calcular a porcentagem de vendas em relação ao total de anúncios e arredondar para duas casas decimais
mkt['%_de_vendas'] = ((mkt['sales'] / mkt['Total_de_anuncios']) * 100).round(2)

mkt.head()

Como podem observar acima, foi adicionado duas colunas para podermos verificar a relação entre o valor total de investimento e a porcentagem de vendas relacionado a campanha investida.

In [ ]:
#informações sobre os dados (tipo de dados). Verificando se precisa tratar os dados.
mkt.info()

In [ ]:
#função para descrever: contagem, mínimo, máximo, desvio padrão e média.
mkt.describe()

In [ ]:
# Selecionar apenas as colunas desejadas
mkt_novo = mkt[['youtube', 'facebook', 'newspaper', 'sales']]

# Calcular a matriz de correlação
correlation_matrix = mkt_novo.corr()

# Plotar a matriz de correlação usando um mapa de calor
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix,annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)
plt.title('Matriz de Correlação')
plt.show()

Vendo o modelo de correlação entre as plataformas de investimentos e a quantidade vendida, pode se observar que youtube e facebook apresentam uma correlação muito grande com o retorno de vendas, sendo que o primeiro tem 0.7 em uma escala de -1 até 1, onde 1 é o nível máximo positivo e -1 o mínimo, e o segundo de 0.69 nesta mesma escala.

In [ ]:
# Plotar a distribuição dos dados usando um pairplot
sns.pairplot(mkt_novo)
plt.title('Distribuição dos Dados')
plt.show()

Como podemos observar através do modelo de distribuição, o youtube se mostra mais uma vez sendo a ferramenta mais eficiente nos anuncios, pois como podem observear não há tanta dispersão dos dados, ou seja, quanto mais aproximado maior é a relação dos mesmos.

In [61]:
#selecionar as features(X) e a target(y):
X = mkt[['youtube','facebook','newspaper']]
y = mkt[['sales']]

#treinar e testar as features e a target:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#criar modelo de regressão linear:
model = LinearRegression()

#treinar modelo:
model.fit(X_train, y_train)

#fazer previsões nos dados de teste:
predictions = model.predict(X_test)

#Avaliar o desempenho do modelo:
mse = mean_squared_error(y_test, predictions).round(4)
print("MSE: ", mse)

# Calcular o coeficiente de determinação (R^2) nos dados de teste
r2_score = model.score(X_test, y_test).round(4)
print("Coeficiente de determinação (R^2):", r2_score)

MSE:  5.5768
Coeficiente de determinação (R^2): 0.8729


In [63]:
# Fazer previsões de vendas com base nos investimentos em marketing
vendas_previstas = model.predict(X_test).round(2)

# Visualizar as vendas previstas
print("Vendas previstas:")
print(vendas_previstas)

Vendas previstas:
[[12.98]
 [25.06]
 [18.24]
 [20.63]
 [11.59]
 [ 8.56]
 [13.35]
 [23.17]
 [12.03]
 [24.96]
 [10.23]
 [13.43]
 [13.98]
 [11.03]
 [20.76]
 [26.45]
 [28.93]
 [21.76]
 [21.25]
 [24.16]
 [20.68]
 [ 9.84]
 [19.43]
 [18.24]
 [14.29]
 [26.11]
 [20.88]
 [10.58]
 [12.22]
 [23.14]
 [ 7.01]
 [ 8.46]
 [16.6 ]
 [21.84]
 [18.14]]
